In [1]:
library(tidyverse)
library(phyloseq)
library(ape)
library(microbiome)

── Attaching packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


microbiome R package (microbiome.github.com)
    


 Copyright (C) 2011-2020 Leo Lahti, 
    Sudarshan Shetty et al. <microbiome.github.io>



Attaching package: ‘microbiome’


The following object is masked from ‘package:ggplot2’:

    alpha


The following object is masked from ‘package:base’:

    transform




In [2]:
otu <- read.delim("../data/otu.GTDB98.txt", sep = "\t", header = TRUE)

In [3]:
tax_map <- read.delim('/mnt/btrfs/data/shogun/gtdb_95/r95.gtdb.tax', sep="\t", header=F)

In [4]:
colnames(tax_map) <- c('X.OTU.ID', 'tax')

In [5]:
otu <- tax_map %>% inner_join(otu, by = 'X.OTU.ID')

rownames(otu)<- otu$X.OTU.ID

In [6]:
taxa <- otu %>% 
  select(tax) %>% 
  separate(tax, c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species"),
           ";")  

In [7]:
#the output is a data frame of characters, and we need taxa to be recognized as factors
taxa<- taxa %>% 
  mutate_if(is.character, as.factor)

# taxa<- cbind(otu$X.OTU.ID, taxa)

# colnames(taxa)[1] <- "X.OTU.ID"

otu<- otu %>% 
  select(-X.OTU.ID, -tax)

# rownames(taxa) <- taxa$X.OTU.ID

# taxa <- taxa %>% 
#   select(-X.OTU.ID)

In [8]:
meta <- read.delim("../data/SampleID_map.txt", sep = "\t", header = TRUE)

rownames(meta) <- meta$X.SampleID

#and delete the first column because it is now redundant
meta<- meta %>% 
        select(-X.SampleID)

ids <- intersect(rownames(meta), colnames(otu))

In [9]:
# otu <- otu %>% rownames_to_column('rn') %>% filter(rn %in% ids)
# rownames(otu) <- otu$rnb
# otu <- otu %>% select(-rn)

In [10]:
otu <- otu %>% select(ids)

Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(ids)` instead of `ids` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.



In [11]:
meta <- meta %>% rownames_to_column('rn') %>% filter(rn %in% ids)
rownames(meta) <- meta$rn
meta <- meta %>% select(-rn)

In [12]:
# taxa <- taxa %>% rownames_to_column('rn') %>% filter(rn %in% ids)
# rownames(taxa) <- taxa$rn
# taxa <- taxa %>% select(-rn)

In [13]:
otu_mat<- as.matrix(otu)
tax_mat<- as.matrix(taxa)

#transform data to phyloseq objects
phylo_OTU<- otu_table(otu_mat, taxa_are_rows = TRUE)
phylo_TAX<- tax_table(tax_mat)
phylo_samples<- sample_data(meta)

tree <- read.tree("../data/shogun.tre")

#and put them in one object
phylo_object<- phyloseq(phylo_OTU, phylo_TAX, phylo_samples, tree_file=tree)

In [14]:
tree


Phylogenetic tree with 31910 tips and 31909 internal nodes.

Tip labels:
  GCF_002286985.1, GCF_003781945.1, GCF_000739595.1, GCF_002727125.1, GCF_000337915.1, GCF_004114995.1, ...
Node labels:
  , d__Archaea, 100, 74, 100, 100:p__Halobacteriota, ...

Rooted; includes branch lengths.

In [15]:
sample_names(phylo_object)    

[1] "MCT.f.0035" "MCT.f.0052" "MCT.f.0069" "MCT.f.0417" "MCT.f.0103"
  [6] "MCT.f.0120" "MCT.f.0137" "MCT.f.0154" "MCT.f.0222" "MCT.f.0256"
 [11] "MCT.f.0273" "MCT.f.0290" "MCT.f.0307" "MCT.f.0375" "MCT.f.0392"
 [16] "MCT.f.0426" "MCT.f.0443" "MCT.f.0460" "MCT.f.0477" "MCT.f.0494"
 [21] "MCT.f.0511" "MCT.f.0528" "MCT.f.0579" "MCT.f.0596" "MCT.f.0613"
 [26] "MCT.f.0002" "MCT.f.0019" "MCT.f.0036" "MCT.f.0053" "MCT.f.0070"
 [31] "MCT.f.0104" "MCT.f.0121" "MCT.f.0138" "MCT.f.0155" "MCT.f.0206"
 [36] "MCT.f.0223" "MCT.f.0291" "MCT.f.0308" "MCT.f.0325" "MCT.f.0359"
 [41] "MCT.f.0376" "MCT.f.0393" "MCT.f.0410" "MCT.f.0427" "MCT.f.0444"
 [46] "MCT.f.0461" "MCT.f.0495" "MCT.f.0512" "MCT.f.0529" "MCT.f.0546"
 [51] "MCT.f.0580" "MCT.f.0597" "MCT.f.0614" "MCT.f.0003" "MCT.f.0020"
 [56] "MCT.f.0037" "MCT.f.0071" "MCT.f.0122" "MCT.f.0139" "MCT.f.0156"
 [61] "MCT.f.0224" "MCT.f.0241" "MCT.f.0258" "MCT.f.0275" "MCT.f.0292"
 [66] "MCT.f.0326" "MCT.f.0360" "MCT.f.0377" "MCT.f.0394" "MCT.f.0411"
 [71] "MCT.f.0428" "MCT.f.0445" "MCT.f.0462" "MCT.f.0513" "MCT.f.0530"
 [76] "MCT.f.0547" "MCT.f.0564" "MCT.f.0581" "MCT.f.0598" "MCT.f.0615"
 [81] "MCT.f.0004" "MCT.f.0038" "MCT.f.0055" "MCT.f.0072" "MCT.f.0089"
 [86] "MCT.f.0106" "MCT.f.0123" "MCT.f.0225" "MCT.f.0259" "MCT.f.0293"
 [91] "MCT.f.0310" "MCT.f.0327" "MCT.f.0344" "MCT.f.0361" "MCT.f.0378"
 [96] "MCT.f.0395" "MCT.f.0412" "MCT.f.0429" "MCT.f.0446" "MCT.f.0463"
[101] "MCT.f.0514" "MCT.f.0531" "MCT.f.0565" "MCT.f.0582" "MCT.f.0599"
[106] "MCT.f.0616" "MCT.f.0039" "MCT.f.0107" "MCT.f.0124" "MCT.f.0141"
[111] "MCT.f.0158" "MCT.f.0209" "MCT.f.0226" "MCT.f.0243" "MCT.f.0260"
[116] "MCT.f.0277" "MCT.f.0294" "MCT.f.0311" "MCT.f.0345" "MCT.f.0362"
[121] "MCT.f.0379" "MCT.f.0396" "MCT.f.0430" "MCT.f.0447" "MCT.f.0464"
[126] "MCT.f.0481" "MCT.f.0515" "MCT.f.0532" "MCT.f.0549" "MCT.f.0566"
[131] "MCT.f.0583" "MCT.f.0600" "MCT.f.0006" "MCT.f.0057" "MCT.f.0074"
[136] "MCT.f.0091" "MCT.f.0108" "MCT.f.0142" "MCT.f.0159" "MCT.f.0227"
[141] "MCT.f.0261" "MCT.f.0295" "MCT.f.0312" "MCT.f.0329" "MCT.f.0346"
[146] "MCT.f.0363" "MCT.f.0380" "MCT.f.0397" "MCT.f.0414" "MCT.f.0431"
[151] "MCT.f.0482" "MCT.f.0516" "MCT.f.0533" "MCT.f.0567" "MCT.f.0601"
[156] "MCT.f.0618" "MCT.f.0007" "MCT.f.0058" "MCT.f.0075" "MCT.f.0109"
[161] "MCT.f.0126" "MCT.f.0143" "MCT.f.0160" "MCT.f.0211" "MCT.f.0228"
[166] "MCT.f.0245" "MCT.f.0313" "MCT.f.0330" "MCT.f.0347" "MCT.f.0364"
[171] "MCT.f.0381" "MCT.f.0415" "MCT.f.0432" "MCT.f.0517" "MCT.f.0534"
[176] "MCT.f.0551" "MCT.f.0568" "MCT.f.0602" "MCT.f.0008" "MCT.f.0042"
[181] "MCT.f.0059" "MCT.f.0076" "MCT.f.0144" "MCT.f.0161" "MCT.f.0229"
[186] "MCT.f.0263" "MCT.f.0297" "MCT.f.0314" "MCT.f.0331" "MCT.f.0348"
[191] "MCT.f.0365" "MCT.f.0382" "MCT.f.0399" "MCT.f.0416" "MCT.f.0433"
[196] "MCT.f.0450" "MCT.f.0467" "MCT.f.0535" "MCT.f.0552" "MCT.f.0586"
[201] "MCT.f.0603" "MCT.f.0620" "MCT.f.0009" "MCT.f.0043" "MCT.f.0060"
[206] "MCT.f.0077" "MCT.f.0094" "MCT.f.0111" "MCT.f.0128" "MCT.f.0145"
[211] "MCT.f.0162" "MCT.f.0230" "MCT.f.0247" "MCT.f.0264" "MCT.f.0298"
[216] "MCT.f.0315" "MCT.f.0349" "MCT.f.0366" "MCT.f.0383" "MCT.f.0400"
[221] "MCT.f.0086" "MCT.f.0434" "MCT.f.0519" "MCT.f.0553" "MCT.f.0570"
[226] "MCT.f.0587" "MCT.f.0604" "MCT.f.0621" "MCT.f.0010" "MCT.f.0044"
[231] "MCT.f.0061" "MCT.f.0095" "MCT.f.0112" "MCT.f.0146" "MCT.f.0163"
[236] "MCT.f.0214" "MCT.f.0231" "MCT.f.0265" "MCT.f.0299" "MCT.f.0316"
[241] "MCT.f.0350" "MCT.f.0384" "MCT.f.0435" "MCT.f.0486" "MCT.f.0520"
[246] "MCT.f.0554" "MCT.f.0605" "MCT.f.0622" "MCT.f.0011" "MCT.f.0045"
[251] "MCT.f.0096" "MCT.f.0113" "MCT.f.0130" "MCT.f.0215" "MCT.f.0232"
[256] "MCT.f.0249" "MCT.f.0266" "MCT.f.0300" "MCT.f.0334" "MCT.f.0351"
[261] "MCT.f.0368" "MCT.f.0385" "MCT.f.0402" "MCT.f.0419" "MCT.f.0453"
[266] "MCT.f.0470" "MCT.f.0538" "MCT.f.0555" "MCT.f.0606" "MCT.f.0623"
[271] "MCT.f.0012" "MCT.f.0063" "MCT.f.0080" "MCT.f.0114" "MCT.f.0148"
[276] "MCT.f.0216" "MCT.f.0233" "MCT.f.0250" "MCT.f.0267" "MCT.f.0301"
[281] "MCT.f.0318" "MCT.f.

In [16]:
phylo_object

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 477 taxa and 415 samples ]
sample_data() Sample Data:       [ 415 samples by 23 sample variables ]
tax_table()   Taxonomy Table:    [ 477 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 477 tips and 476 internal nodes ]

In [17]:
tax_table(phylo_object)

,Domain,Phylum,Class,Order,Family,Genus,Species
GCF_000404225.1,k__Archaea,p__Thermoplasmatota,c__Thermoplasmata,o__Methanomassiliicoccales,f__Methanomassiliicoccaceae,g__Methanomassiliicoccus,s__Methanomassiliicoccus_intestinalis
GCF_000026325.1,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Escherichia,s__Escherichia_coli_D
GCF_003697165.2,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Escherichia,s__Escherichia_coli
GCF_002950215.1,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Escherichia,s__Escherichia_flexneri
GCF_001729805.1,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Enterobacter,s__Enterobacter_roggenkampii
GCF_001750725.1,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Enterobacter,s__Enterobacter_ludwigii
GCF_900200035.1,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Klebsiella_A,s__Klebsiella_A_grimontii
GCF_000191405.1,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Pasteurellaceae,g__Haemophilus_D,s__Haemophilus_D_parainfluenzae
GCF_001815355.1,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Pasteurellaceae,g__Haemophilus_D,s__Haemophilus_D_sp001815355
GCF_000210895.1,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Pasteurellaceae,g__Haemophilus_D,s__Haemophilus_D_parainfluenzae_A


In [18]:
unifrac_weighted = UniFrac(phylo_object, weighted=T, fast=F)

Warning message in UniFrac(phylo_object, weighted = T, fast = F):
“Option `fast=FALSE` is deprecated. Only 'fast' UniFrac is supported in phyloseq.”


In [19]:
unifrac_unweighted = UniFrac(phylo_object, weighted=F, fast=F)

Warning message in UniFrac(phylo_object, weighted = F, fast = F):
“Option `fast=FALSE` is deprecated. Only 'fast' UniFrac is supported in phyloseq.”


In [20]:
write.table(as.matrix(unifrac_weighted), file = "../data/weighted.unifrac.txt", sep = "\t", quote = F, row.names = T)
write.table(as.matrix(unifrac_unweighted), file = "../data/unweighted.unifrac.txt", sep = "\t", quote = F, row.names = T)

In [21]:
alpha(phylo_object)

Observed richness

Other forms of richness

Diversity

Evenness

Dominance

Rarity



,observed,chao1,diversity_inverse_simpson,diversity_gini_simpson,diversity_shannon,diversity_fisher,diversity_coverage,evenness_camargo,evenness_pielou,evenness_simpson,⋯,dominance_dbp,dominance_dmn,dominance_absolute,dominance_relative,dominance_simpson,dominance_core_abundance,dominance_gini,rarity_log_modulo_skewness,rarity_low_abundance,rarity_rare_abundance
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MCT.f.0035,292,292.0000,12.249676,0.9183652,3.577229,32.82235,5,0.17897924,0.6301540,0.04195094,⋯,0.23087703,0.3509725,55353,0.23087703,0.08163481,0.69910032,0.9190019,2.059527,0.09588281,0.11308399
MCT.f.0052,314,314.0000,15.001803,0.9333413,3.659039,35.62320,7,0.33753983,0.6364218,0.04777644,⋯,0.18830424,0.2955640,45145,0.18830424,0.06665865,0.35717533,0.9190603,2.059000,0.08882354,0.32280548
MCT.f.0069,333,348.5741,13.671594,0.9268556,3.376505,38.06476,5,0.17264706,0.5813399,0.04105584,⋯,0.15086108,0.2966962,36170,0.15086108,0.07314436,0.51759490,0.9410301,2.060870,0.08016867,0.21118049
MCT.f.0417,365,434.1364,8.554677,0.8831049,3.181298,42.22315,3,0.08878411,0.5392124,0.02343747,⋯,0.25742137,0.4563477,61716,0.25742137,0.11689512,0.15478400,0.9379981,2.061122,0.09388230,0.72697469
MCT.f.0103,383,407.5000,21.097570,0.9526012,3.793565,44.58596,8,0.13202947,0.6377845,0.05508504,⋯,0.11754401,0.2183211,28182,0.11754401,0.04739882,0.40496002,0.9157124,2.059277,0.08394333,0.35694891
MCT.f.0120,334,359.8065,6.870727,0.8544550,2.966498,38.19403,2,0.09512095,0.5104847,0.02057104,⋯,0.28605452,0.5199668,68581,0.28605452,0.14554501,0.81691610,0.9463791,2.061133,0.08272436,0.10103108
MCT.f.0137,334,334.4310,22.116700,0.9547853,3.754024,38.19366,8,0.11761049,0.6460046,0.06621766,⋯,0.11036594,0.2068350,26462,0.11036594,0.04521470,0.58413203,0.9224552,2.060280,0.08721420,0.23684759
MCT.f.0154,402,451.8462,22.757050,0.9560576,3.957415,47.09837,9,0.12491273,0.6599595,0.05660958,⋯,0.11690316,0.2282381,28028,0.11690316,0.04394243,0.46107677,0.8988067,2.060154,0.09785447,0.20307065
MCT.f.0222,364,364.0000,30.152887,0.9668357,4.204436,42.09189,12,0.17832916,0.7129602,0.08283760,⋯,0.09020803,0.1766291,21629,0.09020803,0.03316432,0.32126889,0.8758264,2.060014,0.10762070,0.47123886


In [22]:
microbiome::alpha(phylo_object)

Observed richness

Other forms of richness

Diversity

Evenness

Dominance

Rarity



,observed,chao1,diversity_inverse_simpson,diversity_gini_simpson,diversity_shannon,diversity_fisher,diversity_coverage,evenness_camargo,evenness_pielou,evenness_simpson,⋯,dominance_dbp,dominance_dmn,dominance_absolute,dominance_relative,dominance_simpson,dominance_core_abundance,dominance_gini,rarity_log_modulo_skewness,rarity_low_abundance,rarity_rare_abundance
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MCT.f.0035,292,292.0000,12.249676,0.9183652,3.577229,32.82235,5,0.17897924,0.6301540,0.04195094,⋯,0.23087703,0.3509725,55353,0.23087703,0.08163481,0.69910032,0.9190019,2.059527,0.09588281,0.11308399
MCT.f.0052,314,314.0000,15.001803,0.9333413,3.659039,35.62320,7,0.33753983,0.6364218,0.04777644,⋯,0.18830424,0.2955640,45145,0.18830424,0.06665865,0.35717533,0.9190603,2.059000,0.08882354,0.32280548
MCT.f.0069,333,348.5741,13.671594,0.9268556,3.376505,38.06476,5,0.17264706,0.5813399,0.04105584,⋯,0.15086108,0.2966962,36170,0.15086108,0.07314436,0.51759490,0.9410301,2.060870,0.08016867,0.21118049
MCT.f.0417,365,434.1364,8.554677,0.8831049,3.181298,42.22315,3,0.08878411,0.5392124,0.02343747,⋯,0.25742137,0.4563477,61716,0.25742137,0.11689512,0.15478400,0.9379981,2.061122,0.09388230,0.72697469
MCT.f.0103,383,407.5000,21.097570,0.9526012,3.793565,44.58596,8,0.13202947,0.6377845,0.05508504,⋯,0.11754401,0.2183211,28182,0.11754401,0.04739882,0.40496002,0.9157124,2.059277,0.08394333,0.35694891
MCT.f.0120,334,359.8065,6.870727,0.8544550,2.966498,38.19403,2,0.09512095,0.5104847,0.02057104,⋯,0.28605452,0.5199668,68581,0.28605452,0.14554501,0.81691610,0.9463791,2.061133,0.08272436,0.10103108
MCT.f.0137,334,334.4310,22.116700,0.9547853,3.754024,38.19366,8,0.11761049,0.6460046,0.06621766,⋯,0.11036594,0.2068350,26462,0.11036594,0.04521470,0.58413203,0.9224552,2.060280,0.08721420,0.23684759
MCT.f.0154,402,451.8462,22.757050,0.9560576,3.957415,47.09837,9,0.12491273,0.6599595,0.05660958,⋯,0.11690316,0.2282381,28028,0.11690316,0.04394243,0.46107677,0.8988067,2.060154,0.09785447,0.20307065
MCT.f.0222,364,364.0000,30.152887,0.9668357,4.204436,42.09189,12,0.17832916,0.7129602,0.08283760,⋯,0.09020803,0.1766291,21629,0.09020803,0.03316432,0.32126889,0.8758264,2.060014,0.10762070,0.47123886


In [23]:
save.image("../data/environment.RData")